# Customer-specialist
Agent responsible of solve fix to customer using requirementes describe on KB procedures

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/Foiac/customerSpecialist/refs/heads/main/customer-specialist/FlowDrawn.png" alt="Clean Data" style="width: 800px">
</div>

### Paths

In [26]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import os
from dotenv import load_dotenv

### Confs

In [33]:
load_dotenv()

# AI SEARCH ENVIRONMENT VARIABLES
SERVICE_NAME = os.getenv("AZURE_SEARCH_SERVICE_NAME")
INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")
API_KEY_AZURE_SEARCH = os.getenv("AZURE_SEARCH_API_KEY")

endpoint = f"https://{SERVICE_NAME}.search.windows.net"

# OPEN AI ENVIRONMENT VARIABLES
AZURE_ENDPOINT = os.getenv("AZURE_OPEN_AI_ENDPOINT")
API_VERSION = os.getenv("AZURE_OPEN_AI_API_VERSION")
API_KEY_AZURE_OPEN_AI = os.getenv("AZURE_OPEN_AI_API_KEY")


### Index Client

In [28]:
search_client = SearchClient(endpoint=endpoint, index_name=INDEX_NAME, credential=AzureKeyCredential(API_KEY_AZURE_SEARCH))

### OpenAI Client

In [35]:
client = AzureOpenAI(
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT,
    api_key=API_KEY_AZURE_OPEN_AI,
)

### Context function

In [36]:
def obter_contexto_rag(pergunta):
    resultados = search_client.search(pergunta, top=3)
    return "\n".join([doc['content'] for doc in resultados])

### Tools

In [37]:
tools = [{
    "type": "function",
    "function": {
        "name": "resetar_senha",
        "description": "Redefine a senha de um colaborador autorizado",
        "parameters": {
            "type": "object",
            "properties": {
                "cpf": {"type": "string"},
                "tipo_usuario": {"type": "string", "enum": ["colaborador", "gerente", "rh"]}
            },
            "required": ["cpf", "tipo_usuario"]
        }
    }
}]

### Define agent function

In [38]:
def chamar_agente(pergunta, contexto):
    resposta = client.chat.completions.create(
        model="gpt-4o-mini", 
        temperature=0,
        tools=tools,
        tool_choice="auto",
        messages=[
            {"role": "system", "content": "Você é um agente de execução que só age com base nos procedimentos."},
            {"role": "user", "content": pergunta},
            {"role": "system", "content": f"Base de conhecimento:\n{contexto}"}
        ]
    )
    return resposta

### Teste

In [39]:
pergunta = "Quero resetar a senha do gerente João, cpf 12345678901"
contexto = obter_contexto_rag(pergunta)

resposta = chamar_agente(pergunta, contexto)

choice = resposta.choices[0]

if choice.finish_reason == "tool_calls":
    print("Decidiu executar:", choice.message.tool_calls[0].function.name)
    print("Parâmetros:", choice.message.tool_calls[0].function.arguments)
else:
    print("Resposta textual:", choice.message.content)

Resposta textual: A redefinição de senha para um gerente, como no caso do João, requer autorização do RH. Você tem essa autorização?
